# Sales Forecasting with Rasgo

This notebook shows how to perform the data preparation and feature engineering for a sales forecasting model. Starting with [AdventureWorks](https://docs.microsoft.com/en-us/sql/samples/adventureworks-install-configure) data preloaded in Rasgo, the data will be explored, features created and modeling data extracted.

This analysis will be focused on the internet sales for this company.

## Packages

The documentation for each packaged used in this tutorial is linked below:
* [numpy](https://numpy.org/doc/stable/)
* [os](https://docs.python.org/3/library/os.html)
* [pandas](https://pandas.pydata.org/docs/)
* [pyrasgo](https://docs.rasgoml.com/rasgo-docs/)
* [scikit-learn](https://scikit-learn.org/stable/)
    * [sklearn.metrics](https://scikit-learn.org/stable/modules/model_evaluation.html)
* [XGBoost](https://xgboost.readthedocs.io/en/latest/)

Uncomment the cell below if you're missing some of these packages on your local machine. 

In [ ]:
#!pip install numpy pandas pyrasgo scikit-learn xgboost

### Ensure that pyrasgo is installed and up to date

In [1]:
!pip install pyrasgo --upgrade

!pip show pyrasgo

     |████████████████████████████████| 107 kB 5.3 MB/s            
  Attempting uninstall: pyrasgo
    Found existing installation: pyrasgo 0.4.9
    Uninstalling pyrasgo-0.4.9:
      Successfully uninstalled pyrasgo-0.4.9
Name: pyrasgo
Version: 0.4.12
Summary: Alpha version of the Rasgo Python interface.
Home-page: https://www.rasgoml.com/
Author: Patrick Dougherty
Author-email: patrick@rasgoml.com
License: GNU Affero General Public License v3 or later (AGPLv3+)
Location: /Users/nick/Git/workspace/venv/lib/python3.9/site-packages
Requires: idna, more-itertools, pandas, pyarrow, pydantic, pyyaml, requests, snowflake-connector-python, snowflake-connector-python, tqdm
Required-by: 


In [3]:
import numpy as np
import os
import pandas as pd
import 
from sklearn.metrics import mean_squared_error
import           as xgb

## Create account on Rasgo

## Access Rasgo

### Create account

Next, click [here](https://app.rasgoml.com/account/register) to create an account on the Rasgo UI. Fill in the required information on the web page.

<p align="center">
  <img src="img/RasgoAccountRegistration.png" alt="Rasgo Account Registration" width="512">
</p>

You can close the browser tab as you will receive an email from rasgo to verify your email address. Click the **Verify Email** button to verify.

<p align="center">
  <img src="img/RasgoWelcome.png" alt="Verify Email" width="390">
</p>

This will open browser tab where you can log into the UI.

### Log into Rasgo UI

Enter your username and password and click **Login**.

<p align="center">
  <img src="img/RasgoLogin.png" alt="Login to Rasgo" width="528">
</p>

to be taken to the Rasgo App homepage.

### Copy your API Key

Click the **API KEY** button in the upper right of the screen

<img src="img/APIKEY.png" alt="Copy API Key" width="128">

to copy your API key to the clipboard.

## Work with PyRasgo

### Load the API Key from the environment variable and connect to the Data Warehouse

In [5]:
rasgo = pyrasgo.          (<your_api_key>)

### Get list of available datasets

Loop over all available datasets and print out the dataset ID and Name.

In [6]:
datasets = sorted(rasgo.get.           (), key=lambda x: x._api_dataset.id)
for ds in datasets:
    print(f"ID: {ds.id}\tDataset: {ds.name}")

ID: 52	Dataset: Adventureworks: Dim Account
ID: 53	Dataset: Adventureworks: Dim Currency
ID: 54	Dataset: Adventureworks: Dim Salesreason
ID: 55	Dataset: Adventureworks: Dim Customer
ID: 56	Dataset: Adventureworks: Dim Promotion
ID: 57	Dataset: Adventureworks: Dim Date
ID: 58	Dataset: Adventureworks: Dim Productcategory
ID: 59	Dataset: Adventureworks: Dim Departmentgroup
ID: 60	Dataset: Adventureworks: Newfact Currencyrate
ID: 61	Dataset: Adventureworks: Dim Geography
ID: 62	Dataset: Adventureworks: Dim Organization
ID: 63	Dataset: Adventureworks: Dim Reseller
ID: 64	Dataset: Adventureworks: Fact Additionalinternationalproductdescription
ID: 65	Dataset: Adventureworks: Fact Productinventory
ID: 66	Dataset: Adventureworks: Fact Callcenter
ID: 67	Dataset: Adventureworks: Fact Resellersales
ID: 68	Dataset: Adventureworks: Fact Internetsalesreason
ID: 69	Dataset: Adventureworks: Fact Salesquota
ID: 70	Dataset: Adventureworks: Prospectivebuyer
ID: 71	Dataset: Adventureworks: Dim Scenario
ID:

The two datasets of interest are 74 containing the sales information and 56 containing details about any promotions in place.

### Examine Internet Sales

In [7]:
internet_sales = rasgo.get.dataset(   )
internet_sales.preview()

,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,PROMOTIONKEY,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,SALESORDERLINENUMBER,...,PRODUCTSTANDARDCOST,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE
0,310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-29,2011-01-10,2011-01-05
1,346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05
2,346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05
3,336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,...,413.1463,413.1463,699.0982,55.9279,17.4775,None,None,2010-12-29,2011-01-10,2011-01-05
4,346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05
5,311,20101230,20110111,20110106,27645,1,100,4,SO43702,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-30,2011-01-11,2011-01-06
6,310,20101230,20110111,20110106,16624,1,6,9,SO43703,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-30,2011-01-11,2011-01-06
7,351,20101230,20110111,20110106,11005,1,6,9,SO43704,1,...,1898.0944,1898.0944,3374.9900,269.9992,84.3748,None,None,2010-12-30,2011-01-11,2011-01-06
8,344,20101230,20110111,20110106,11011,1,6,9,SO43705,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-30,2011-01-11,2011-01-06
9,312,20101231,20110112,20110107,27621,1,100,4,SO43706,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-31,2011-01-12,2011-01-07


This looks promising, but I'd like to see a single product sorted by date. This can be done through the use of the filter and order transforms. To use filter, the product we want to filter on is needed, as we don't know that yet, we will just order by *PRODUCTKEY* and *ORDERDATE*.

In [10]:
internet_sales.order(order_by={'PRODUCTKEY':'ASC',             :'ASC'}).preview()

,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,PROMOTIONKEY,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,SALESORDERLINENUMBER,...,PRODUCTSTANDARDCOST,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE
0,214,20121228,20130109,20130104,12132,1,100,7,SO51181,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04
1,214,20121228,20130109,20130104,16313,1,100,8,SO51180,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04
2,214,20121229,20130110,20130105,12390,1,100,8,SO51191,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05
3,214,20121229,20130110,20130105,11241,1,100,7,SO51192,2,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05
4,214,20121230,20130111,20130106,11338,1,100,8,SO51207,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06
5,214,20121230,20130111,20130106,24604,1,6,9,SO51212,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06
6,214,20121231,20130112,20130107,11061,1,6,9,SO51237,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
7,214,20121231,20130112,20130107,25625,1,100,8,SO51246,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
8,214,20121231,20130112,20130107,11615,1,98,10,SO51232,2,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
9,214,20121231,20130112,20130107,28204,1,6,9,SO51234,2,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07


This looks reasonable, use this for our modeling. For future reference, what columns exist in this table?

In [11]:
internet_sales.preview().columns.sort_values()

Index(['CARRIERTRACKINGNUMBER', 'CURRENCYKEY', 'CUSTOMERKEY',
       'CUSTOMERPONUMBER', 'DISCOUNTAMOUNT', 'DUEDATE', 'DUEDATEKEY',
       'EXTENDEDAMOUNT', 'FREIGHT', 'ORDERDATE', 'ORDERDATEKEY',
       'ORDERQUANTITY', 'PRODUCTKEY', 'PRODUCTSTANDARDCOST', 'PROMOTIONKEY',
       'REVISIONNUMBER', 'SALESAMOUNT', 'SALESORDERLINENUMBER',
       'SALESORDERNUMBER', 'SALESTERRITORYKEY', 'SHIPDATE', 'SHIPDATEKEY',
       'TAXAMT', 'TOTALPRODUCTCOST', 'UNITPRICE', 'UNITPRICEDISCOUNTPCT'],
      dtype='object')

*PROMOTIONKEY* is probably important for a sales forecast. Promotion information can be found in dataset 56.

In [12]:
promotion = rasgo.get.dataset(   )
promotion.preview()

,PROMOTIONKEY,PROMOTIONALTERNATEKEY,ENGLISHPROMOTIONNAME,SPANISHPROMOTIONNAME,FRENCHPROMOTIONNAME,DISCOUNTPCT,ENGLISHPROMOTIONTYPE,SPANISHPROMOTIONTYPE,FRENCHPROMOTIONTYPE,ENGLISHPROMOTIONCATEGORY,SPANISHPROMOTIONCATEGORY,FRENCHPROMOTIONCATEGORY,STARTDATE,ENDDATE,MINQTY,MAXQTY
0,1,1,No Discount,Sin descuento,Aucune remise,0.00,No Discount,Sin descuento,Aucune remise,No Discount,Sin descuento,Aucune remise,2010-11-29,2014-06-30,0,NaN
1,2,2,Volume Discount 11 to 14,Descuento por volumen (entre 11 y 14),Remise sur quantité (de 11 à 14),0.02,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,11,14.0
2,3,3,Volume Discount 15 to 24,Descuento por volumen (entre 15 y 24),Remise sur quantité (de 15 à 24),0.05,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,15,24.0
3,4,4,Volume Discount 25 to 40,Descuento por volumen (entre 25 y 40),Remise sur quantité (de 25 à 40),0.10,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,25,40.0
4,5,5,Volume Discount 41 to 60,Descuento por volumen (entre 41 y 60),Remise sur quantité (de 41 à 60),0.15,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,41,60.0
5,6,6,Volume Discount over 60,Descuento por volumen (más de 60),Remise sur quantité (au-delà de 60),0.20,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,61,NaN
6,7,7,Mountain-100 Clearance Sale,"Liquidación de bicicleta de montaña, 100",Liquidation VTT 100,0.35,Discontinued Product,Descatalogado,Ce produit n'est plus commercialisé,Reseller,Distribuidor,Revendeur,2011-11-12,2011-12-28,0,NaN
7,8,8,Sport Helmet Discount-2002,"Casco deportivo, descuento: 2002",Remise sur les casques sport - 2002,0.10,Seasonal Discount,Descuento de temporada,Remise saisonnière,Reseller,Distribuidor,Revendeur,2011-12-29,2012-01-28,0,NaN
8,9,9,Road-650 Overstock,"Bicicleta de carretera: 650, oferta especial",Déstockage Vélo de route 650,0.30,Excess Inventory,Inventario excedente,Déstockage,Reseller,Distribuidor,Revendeur,2011-12-29,2012-02-28,0,NaN
9,10,10,Mountain Tire Sale,Oferta de cubierta de montaña,Vente de pneus de VTT,0.50,Excess Inventory,Inventario excedente,Déstockage,Customer,Cliente,Client,2012-12-12,2013-02-26,0,NaN


## Sales Data

Work with the sales and promotion data to create the base modeling time-series features for the sales forecasting model.

### Merge Promo data

First, we want to clean up the promotion data to only keep what needs to be added to the sales data. Drop all columns except *PROMOTIONKEY* and *DISCOUNTPCT* from  promotion using the `drop_columns` transformation.

In [94]:
reduced_promo = promotion.drop_columns(include_cols=[                , 'DISCOUNTPCT'])
reduced_promo.order(order_by={'PROMOTIONKEY':'ASC'}).preview()

,PROMOTIONKEY,DISCOUNTPCT
0,1,0.00
1,2,0.02
2,3,0.05
3,4,0.10
4,5,0.15
5,6,0.20
6,7,0.35
7,8,0.10
8,9,0.30
9,10,0.50


Now merge this with the internet sales datausing the `join` transformation.

In [95]:
sales_promo = reduced_promo.join(join_table=internet_sales,
                                 join_type=       ,
                                 join_columns={'PROMOTIONKEY':'PROMOTIONKEY'})

sales_promo.order(order_by={'PRODUCTKEY':'ASC', 'ORDERDATE':'ASC'}).preview()

,DISCOUNTPCT,PROMOTIONKEY,REVISIONNUMBER,SHIPDATE,TOTALPRODUCTCOST,CUSTOMERPONUMBER,UNITPRICEDISCOUNTPCT,DUEDATEKEY,EXTENDEDAMOUNT,PRODUCTKEY,...,CARRIERTRACKINGNUMBER,ORDERDATE,SALESTERRITORYKEY,CUSTOMERKEY,DUEDATE,ORDERQUANTITY,PRODUCTSTANDARDCOST,CURRENCYKEY,UNITPRICE,SHIPDATEKEY
0,0.0,1,1,2013-01-04,13.0863,None,0.0,20130109,34.99,214,...,None,2012-12-28,7,12132,2013-01-09,1,13.0863,100,34.99,20130104
1,0.0,1,1,2013-01-04,13.0863,None,0.0,20130109,34.99,214,...,None,2012-12-28,8,16313,2013-01-09,1,13.0863,100,34.99,20130104
2,0.0,1,1,2013-01-05,13.0863,None,0.0,20130110,34.99,214,...,None,2012-12-29,7,11241,2013-01-10,1,13.0863,100,34.99,20130105
3,0.0,1,1,2013-01-05,13.0863,None,0.0,20130110,34.99,214,...,None,2012-12-29,8,12390,2013-01-10,1,13.0863,100,34.99,20130105
4,0.0,1,1,2013-01-06,13.0863,None,0.0,20130111,34.99,214,...,None,2012-12-30,8,11338,2013-01-11,1,13.0863,100,34.99,20130106
5,0.0,1,1,2013-01-06,13.0863,None,0.0,20130111,34.99,214,...,None,2012-12-30,9,24604,2013-01-11,1,13.0863,6,34.99,20130106
6,0.0,1,1,2013-01-07,13.0863,None,0.0,20130112,34.99,214,...,None,2012-12-31,9,11061,2013-01-12,1,13.0863,6,34.99,20130107
7,0.0,1,1,2013-01-07,13.0863,None,0.0,20130112,34.99,214,...,None,2012-12-31,9,28204,2013-01-12,1,13.0863,6,34.99,20130107
8,0.0,1,1,2013-01-07,13.0863,None,0.0,20130112,34.99,214,...,None,2012-12-31,8,25625,2013-01-12,1,13.0863,100,34.99,20130107
9,0.0,1,1,2013-01-07,13.0863,None,0.0,20130112,34.99,214,...,None,2012-12-31,10,11615,2013-01-12,1,13.0863,98,34.99,20130107


### Create Weekly Data

Now, we want to forecast these sales weekly, so we need to extract the week from the *ORDERDATE*. This can be done using the transform `datetrunc`.

In [21]:
salesds = sales_promo.datetrunc(dates={'ORDERDATE':       })
salesds.order(order_by={'PRODUCTKEY':'ASC', 'ORDERDATE':'ASC'}).preview()

,PROMOTIONKEY,DISCOUNTPCT,REVISIONNUMBER,SHIPDATE,TOTALPRODUCTCOST,ORDERDATEKEY,DISCOUNTAMOUNT,FREIGHT,CARRIERTRACKINGNUMBER,ORDERDATE,...,SALESORDERLINENUMBER,SALESORDERNUMBER,CUSTOMERKEY,DUEDATE,ORDERQUANTITY,PRODUCTSTANDARDCOST,CURRENCYKEY,UNITPRICE,SHIPDATEKEY,ORDERDATE_WEEK
0,1,0.0,1,2013-01-04,13.0863,20121228,0.0,0.8748,None,2012-12-28,...,4,SO51181,12132,2013-01-09,1,13.0863,100,34.99,20130104,2012-12-23
1,1,0.0,1,2013-01-04,13.0863,20121228,0.0,0.8748,None,2012-12-28,...,4,SO51180,16313,2013-01-09,1,13.0863,100,34.99,20130104,2012-12-23
2,1,0.0,1,2013-01-05,13.0863,20121229,0.0,0.8748,None,2012-12-29,...,2,SO51192,11241,2013-01-10,1,13.0863,100,34.99,20130105,2012-12-23
3,1,0.0,1,2013-01-05,13.0863,20121229,0.0,0.8748,None,2012-12-29,...,4,SO51191,12390,2013-01-10,1,13.0863,100,34.99,20130105,2012-12-23
4,1,0.0,1,2013-01-06,13.0863,20121230,0.0,0.8748,None,2012-12-30,...,4,SO51207,11338,2013-01-11,1,13.0863,100,34.99,20130106,2012-12-30
5,1,0.0,1,2013-01-06,13.0863,20121230,0.0,0.8748,None,2012-12-30,...,4,SO51212,24604,2013-01-11,1,13.0863,6,34.99,20130106,2012-12-30
6,1,0.0,1,2013-01-07,13.0863,20121231,0.0,0.8748,None,2012-12-31,...,4,SO51237,11061,2013-01-12,1,13.0863,6,34.99,20130107,2012-12-30
7,1,0.0,1,2013-01-07,13.0863,20121231,0.0,0.8748,None,2012-12-31,...,2,SO51234,28204,2013-01-12,1,13.0863,6,34.99,20130107,2012-12-30
8,1,0.0,1,2013-01-07,13.0863,20121231,0.0,0.8748,None,2012-12-31,...,4,SO51246,25625,2013-01-12,1,13.0863,100,34.99,20130107,2012-12-30
9,1,0.0,1,2013-01-07,13.0863,20121231,0.0,0.8748,None,2012-12-31,...,2,SO51232,11615,2013-01-12,1,13.0863,98,34.99,20130107,2012-12-30


The new week column is called *ORDERDATE_WEEK*. This is clunky, so let's rename it to *ORDERWEEK* using the `rename` transformation.

In [ ]:
# newsalesds = salesds.rename(renames={'ORDERDATE_WEEK': 'ORDERWEEK'})
# newsalesds.order(order_by={'PRODUCTKEY':'ASC', 'ORDERDATE':'ASC'}).preview()

Alternatively, we can just chain these transformations together.

In [27]:
salesds = sales_promo.           (dates={'ORDERDATE': 'week'}).        (
                                renames={'ORDERDATE_WEEK': 'ORDERWEEK'})
salesds.order(order_by={'PRODUCTKEY':'ASC', 'ORDERWEEK':'ASC'}).preview()

,ORDERWEEK,SALESORDERLINENUMBER,ORDERQUANTITY,CUSTOMERKEY,EXTENDEDAMOUNT,REVISIONNUMBER,CURRENCYKEY,DISCOUNTAMOUNT,FREIGHT,TAXAMT,...,ORDERDATEKEY,SALESORDERNUMBER,CUSTOMERPONUMBER,PRODUCTSTANDARDCOST,TOTALPRODUCTCOST,UNITPRICEDISCOUNTPCT,CARRIERTRACKINGNUMBER,DISCOUNTPCT,SHIPDATE,SHIPDATEKEY
0,2012-12-23,2,1,11241,34.99,1,100,0.0,0.8748,2.7992,...,20121229,SO51192,None,13.0863,13.0863,0.0,None,0.0,2013-01-05,20130105
1,2012-12-23,4,1,12390,34.99,1,100,0.0,0.8748,2.7992,...,20121229,SO51191,None,13.0863,13.0863,0.0,None,0.0,2013-01-05,20130105
2,2012-12-23,4,1,16313,34.99,1,100,0.0,0.8748,2.7992,...,20121228,SO51180,None,13.0863,13.0863,0.0,None,0.0,2013-01-04,20130104
3,2012-12-23,4,1,12132,34.99,1,100,0.0,0.8748,2.7992,...,20121228,SO51181,None,13.0863,13.0863,0.0,None,0.0,2013-01-04,20130104
4,2012-12-30,4,1,24604,34.99,1,6,0.0,0.8748,2.7992,...,20121230,SO51212,None,13.0863,13.0863,0.0,None,0.0,2013-01-06,20130106
5,2012-12-30,4,1,11338,34.99,1,100,0.0,0.8748,2.7992,...,20121230,SO51207,None,13.0863,13.0863,0.0,None,0.0,2013-01-06,20130106
6,2012-12-30,4,1,15553,34.99,1,100,0.0,0.8748,2.7992,...,20130105,SO51309,None,13.0863,13.0863,0.0,None,0.0,2013-01-12,20130112
7,2012-12-30,5,1,11456,34.99,1,6,0.0,0.8748,2.7992,...,20130102,SO51267,None,13.0863,13.0863,0.0,None,0.0,2013-01-09,20130109
8,2012-12-30,2,1,18544,34.99,1,19,0.0,0.8748,2.7992,...,20130101,SO51258,None,13.0863,13.0863,0.0,None,0.0,2013-01-08,20130108
9,2012-12-30,4,1,11433,34.99,1,100,0.0,0.8748,2.7992,...,20130101,SO51259,None,13.0863,13.0863,0.0,None,0.0,2013-01-08,20130108


Now we can aggregate this to the product-week level and create aggregations of the *'DISCOUNTAMOUNT'*, *'DISCOUNTPCT'*, *'ORDERQUANTITY'*, *'PRODUCTSTANDARDCOST'*, *'SALESAMOUNT'*, *'TAXAMT'*, *'TOTALPRODUCTCOST'*, *'UNITPRICE'*, *'UNITPRICEDISCOUNTPCT'* using the `aggregate` transform.

In [43]:
salesds_agg = sales_promo.datetrunc(dates={'ORDERDATE': 'week'}).rename(
                                renames={'ORDERDATE_WEEK': 'ORDERWEEK'}).aggregate(
                                group_by=['PRODUCTKEY', 'ORDERWEEK'],
                                aggregations={'DISCOUNTAMOUNT': [        ,        ,        ,        ], 
                                              'DISCOUNTPCT': ['MIN','MAX', 'AVG', 'SUM'],
                                              'ORDERQUANTITY': ['SUM'],
                                              'PRODUCTSTANDARDCOST': ['AVG', 'SUM'],
                                              'SALESAMOUNT': ['SUM'], 
                                              'TAXAMT': ['SUM'],
                                              'TOTALPRODUCTCOST': ['AVG', 'SUM'],
                                              'UNITPRICE': ['AVG', 'SUM'],
                                              'UNITPRICEDISCOUNTPCT': ['MIN', 'MAX', 'AVG', 'SUM']})
salesds_agg.describe().to_df()

,FEATURE,DTYPE,COUNT,NULL_COUNT,UNIQUE_COUNT,MOST_FREQUENT,MEAN,STD_DEV,MIN,_25_PERCENTILE,_50_PERCENTILE,_75_PERCENTILE,MAX
0,DISCOUNTAMOUNT_AVG,FLOAT,6935,1,1,0,0.000000,0.000000,0,0.0000,0.0000,0.0000,0
1,UNITPRICEDISCOUNTPCT_SUM,FLOAT,6935,1,1,0,0.000000,0.000000,0,0.0000,0.0000,0.0000,0
2,PRODUCTSTANDARDCOST_AVG,NUMBER,6935,1,45,1481.9379000000,661.195888,651.412800,0.8565000000,26.1763,461.4448,1251.9813,2171.2942000000
3,DISCOUNTPCT_AVG,FLOAT,6936,0,98,0,0.001435,0.009993,0,0.0000,0.0000,0.0000,0.2625
4,DISCOUNTPCT_MAX,FLOAT,6936,0,5,0,0.002602,0.013986,0,0.0000,0.0000,0.0000,0.5
5,ORDERQUANTITY_SUM,NUMBER,6935,1,97,1,8.709156,13.494000,1,2.0000,4.0000,8.0000,105
6,PRODUCTKEY,NUMBER,6935,1,158,477,443.479740,111.243992,214,355.0000,472.0000,560.0000,606
7,SALESAMOUNT_SUM,NUMBER,6935,1,772,782.9900,4233.406953,6276.496610,4.9900,524.7900,1484.7000,5102.9700,46517.5100
8,TOTALPRODUCTCOST_AVG,NUMBER,6935,1,45,1481.9379000000,661.195888,651.412800,0.8565000000,26.1763,461.4448,1251.9813,2171.2942000000
9,UNITPRICE_AVG,NUMBER,6935,1,42,2443.3500000000,1110.230001,1092.563872,2.2900000000,53.9900,769.4900,2181.5625,3578.2700000000


This gives us statistics for each product over a given week.

### Time-series feature engineering

For sales forcasting, in addition to the lagged variables, we need to know what the sales were in prior weeks. The transform `lag` can create these variables for us. In this case we will lag the following variables *'DISCOUNTAMOUNT_AVG'*, *'DISCOUNTPCT_AVG'*, *'ORDERQUANTITY_SUM'*, *'PRODUCTSTANDARDCOST_AVG'*, *'SALESAMOUNT_SUM'*, *'TAXAMT_SUM'*, *'TOTALPRODUCTCOST_SUM'*,*'UNITPRICEDISCOUNTPCT_AVG'*, *'UNITPRICE_AVG'*, *'UNITPRICE_SUM'*
over *1*, *2*, *3*, and *12* weeks.

In [52]:
salesds = salesds_agg.lag(columns=['DISCOUNTAMOUNT_AVG', 'DISCOUNTPCT_AVG', 'ORDERQUANTITY_SUM', 
                                         'PRODUCTSTANDARDCOST_AVG', 'SALESAMOUNT_SUM', 'TAXAMT_SUM', 
                                         'TOTALPRODUCTCOST_SUM','UNITPRICEDISCOUNTPCT_AVG', 
                                         'UNITPRICE_AVG', 'UNITPRICE_SUM'],
                                amounts=[1,   , 3,   ],
                                order_by=['PRODUCTKEY', 'ORDERWEEK'],
                                partition=['PRODUCTKEY'])
   
salesds.order(order_by={'PRODUCTKEY':'ASC', 'ORDERWEEK':'ASC'}).preview()

,FEATURE,DTYPE,COUNT,NULL_COUNT,UNIQUE_COUNT,MOST_FREQUENT,MEAN,STD_DEV,MIN,_25_PERCENTILE,_50_PERCENTILE,_75_PERCENTILE,MAX
0,DISCOUNTPCT_MAX,FLOAT,6936,0,5,0,0.002602,0.013986,0,0.0000,0.000,0.0000,0.5
1,LAG_DISCOUNTAMOUNT_AVG_1,FLOAT,6777,159,1,0,0.000000,0.000000,0,0.0000,0.000,0.0000,0
2,LAG_DISCOUNTPCT_AVG_1,FLOAT,6777,159,97,0,0.001415,0.009587,0,0.0000,0.000,0.0000,0.2
3,LAG_SALESAMOUNT_SUM_3,NUMBER,6461,475,751,782.9900,4196.005478,6203.203670,4.9900,521.8200,1484.700,5002.1875,46517.5100
4,LAG_UNITPRICE_AVG_2,NUMBER,6619,317,42,2443.3500000000,1108.667236,1093.999055,2.2900000000,53.9900,769.490,2181.5625,3578.2700000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,LAG_DISCOUNTPCT_AVG_2,FLOAT,6619,317,96,0,0.001432,0.009688,0,0.0000,0.000,0.0000,0.2
59,LAG_TAXAMT_SUM_1,NUMBER,6777,159,765,62.6392,338.294394,501.399249,0.3992,41.9832,118.776,408.2376,3721.4008
60,UNITPRICE_AVG,NUMBER,6935,1,42,2443.3500000000,1110.230001,1092.563872,2.2900000000,53.9900,769.490,2181.5625,3578.2700000000
61,LAG_UNITPRICE_SUM_2,NUMBER,6619,317,762,782.9900,4210.776581,6233.496379,4.9900,521.8200,1484.700,5002.1875,46517.5100


In addition to lag variables, the moving average of the quantites can be useful. In this case, we'll calculate the moving average over *4* observations of *ORDERQUANTITY_SUM* and *TOTALPRODUCTCOST_SUM* using the transform `moving_avg`.

In [53]:
salesds = salesds.moving_avg(input_columns=['ORDERQUANTITY_SUM', 'SALESAMOUNT_SUM'],
                             window_sizes=[   ],
                             order_by=['PRODUCTKEY', 'ORDERWEEK'],
                             partition=['PRODUCTKEY'])
    
salesds.order(order_by={'PRODUCTKEY':'ASC', 'ORDERWEEK':'ASC'}).preview()

,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,DISCOUNTAMOUNT_MAX,DISCOUNTAMOUNT_AVG,DISCOUNTAMOUNT_SUM,DISCOUNTPCT_MIN,DISCOUNTPCT_MAX,DISCOUNTPCT_AVG,DISCOUNTPCT_SUM,...,LAG_UNITPRICE_AVG_1,LAG_UNITPRICE_AVG_2,LAG_UNITPRICE_AVG_3,LAG_UNITPRICE_AVG_12,LAG_UNITPRICE_SUM_1,LAG_UNITPRICE_SUM_2,LAG_UNITPRICE_SUM_3,LAG_UNITPRICE_SUM_12,MEAN_ORDERQUANTITY_SUM_4,MEAN_SALESAMOUNT_SUM_4
0,214,2012-12-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,4.000,139.9600000
1,214,2012-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,None,None,None,139.9600,None,None,None,7.000,244.9300000
2,214,2013-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,None,None,349.9000,139.9600,None,None,8.666,303.2466666
3,214,2013-01-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,419.8800,349.9000,139.9600,None,8.000,279.9200000
4,214,2013-01-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,209.9400,419.8800,349.9000,None,9.750,341.1525000
5,214,2013-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,384.8900,209.9400,419.8800,None,15.000,524.8500000
6,214,2013-02-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1084.6900,384.8900,209.9400,None,23.000,804.7700000
7,214,2013-02-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1539.5600,1084.6900,384.8900,None,30.000,1049.7000000
8,214,2013-02-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1189.6600,1539.5600,1084.6900,None,35.500,1242.1450000
9,214,2013-02-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1154.6700,1189.6600,1539.5600,None,37.000,1294.6300000


#### Save result

At this point, the data has been aggregated to weekly data and multiple transformations have been applied. This could be a good starting point for additional analysis and useful for visualization. For this reason, we will publish it to Rasgo to make it available for others to use. This can be done with the `rasgo.publish.dataset` function

In [54]:
weeklysales = rasgo.publish.dataset(dataset=        ,
                                    name="WKSP: AdventureWorks: weekly sales",
                                    description="Internet Sales data converted to weekly sales")
weeklysales

Dataset(id=2389, name=WKSP: AdventureWorks: weekly sales, status=published, description=Internet Sales data converted to weekly sales)

We can examine this dataset on Rasgo by clicking the link below

In [55]:
print(f"https://app.rasgoml.com/datasets/{weeklysales.id}")

https://app.rasgoml.com/datasets/2389


Using this dataset, we can continue data preparation.

### Create Modeling Data

We can now begin final preparation for modeling with this dataset. To do this, we need to do three things. First, the target (next weeks sales needs to be created). Second, the categorical variables should be one-hot encoded. Finally, missing values should be imputed for the numeric columns.

#### Target Creation

Use the `lag` transform with a negative lag value to get next weeks sales as the target. While doing this, rename the value to make it clear that it is the target.

In [64]:
modelingds = weeklysales.lag(columns=['SALESAMOUNT_SUM'],
                             amounts=[   ],
                             order_by=['PRODUCTKEY', 'ORDERWEEK'],
                             partition =['PRODUCTKEY']).rename(
                             renames={'LAG_SALESAMOUNT_SUM__1': 'TARGET_SALESAMOUNT'})
modelingds.preview().columns.sort_values()

Index(['DISCOUNTAMOUNT_AVG', 'DISCOUNTAMOUNT_MAX', 'DISCOUNTAMOUNT_MIN',
       'DISCOUNTAMOUNT_SUM', 'DISCOUNTPCT_AVG', 'DISCOUNTPCT_MAX',
       'DISCOUNTPCT_MIN', 'DISCOUNTPCT_SUM', 'LAG_DISCOUNTAMOUNT_AVG_1',
       'LAG_DISCOUNTAMOUNT_AVG_12', 'LAG_DISCOUNTAMOUNT_AVG_2',
       'LAG_DISCOUNTAMOUNT_AVG_3', 'LAG_DISCOUNTPCT_AVG_1',
       'LAG_DISCOUNTPCT_AVG_12', 'LAG_DISCOUNTPCT_AVG_2',
       'LAG_DISCOUNTPCT_AVG_3', 'LAG_ORDERQUANTITY_SUM_1',
       'LAG_ORDERQUANTITY_SUM_12', 'LAG_ORDERQUANTITY_SUM_2',
       'LAG_ORDERQUANTITY_SUM_3', 'LAG_PRODUCTSTANDARDCOST_AVG_1',
       'LAG_PRODUCTSTANDARDCOST_AVG_12', 'LAG_PRODUCTSTANDARDCOST_AVG_2',
       'LAG_PRODUCTSTANDARDCOST_AVG_3', 'LAG_SALESAMOUNT_SUM_1',
       'LAG_SALESAMOUNT_SUM_12', 'LAG_SALESAMOUNT_SUM_2',
       'LAG_SALESAMOUNT_SUM_3', 'LAG_TAXAMT_SUM_1', 'LAG_TAXAMT_SUM_12',
       'LAG_TAXAMT_SUM_2', 'LAG_TAXAMT_SUM_3', 'LAG_TOTALPRODUCTCOST_SUM_1',
       'LAG_TOTALPRODUCTCOST_SUM_12', 'LAG_TOTALPRODUCTCOST_SUM_2',
  

#### Categorical encoding

The only column that needs to be encoded *PRODUCTKEY*. We can use the `one_hot_encode`, `target_encode` or the `label_encode` transforms. 

For features with a large number of categorties and when using tree-based modeling algorithms, the `label_encode` transform is a useful technique to encode categorical variables. The `target_encode` transform replaces the category by the mean target value of that category. Target encoding is a very powerful techinque to encode these high-cardinality categorical variables efficiently and help improve model performance. In this case, we will target encode the product_key

In [66]:
modelingds_encoded = modelingds.target_encode(column='PRODUCTKEY',
                                      target=                      )

modelingds_encoded.order(order_by={'PRODUCTKEY':'ASC', 'ORDERWEEK':'ASC'}).preview()

,TARGET_SALESAMOUNT,LAG_SALESAMOUNT_SUM_2,LAG_TAXAMT_SUM_1,MEAN_SALESAMOUNT_SUM_4,UNITPRICEDISCOUNTPCT_AVG,LAG_PRODUCTSTANDARDCOST_AVG_1,DISCOUNTAMOUNT_MAX,DISCOUNTPCT_MAX,LAG_DISCOUNTAMOUNT_AVG_2,LAG_DISCOUNTPCT_AVG_1,...,LAG_UNITPRICE_AVG_1,LAG_UNITPRICE_SUM_2,UNITPRICEDISCOUNTPCT_MIN,UNITPRICE_AVG,LAG_DISCOUNTPCT_AVG_2,LAG_TOTALPRODUCTCOST_SUM_2,LAG_UNITPRICEDISCOUNTPCT_AVG_2,ORDERQUANTITY_SUM,LAG_SALESAMOUNT_SUM_3,PRODUCTKEY_TARGET_ENCODED
0,349.9000,None,NaN,139.9600000,0.0,None,0.0,0.0,NaN,NaN,...,None,None,0.0,34.9900000000,NaN,None,NaN,4,None,1366.452
1,419.8800,None,11.1968,244.9300000,0.0,13.0863000000,0.0,0.0,NaN,0.0,...,34.9900000000,None,0.0,34.9900000000,NaN,None,NaN,10,None,1366.452
2,209.9400,139.9600,27.9920,303.2466666,0.0,13.0863000000,0.0,0.0,0.0,0.0,...,34.9900000000,139.9600,0.0,34.9900000000,0.0,52.3452,0.0,12,None,1366.452
3,384.8900,349.9000,33.5904,279.9200000,0.0,13.0863000000,0.0,0.0,0.0,0.0,...,34.9900000000,349.9000,0.0,34.9900000000,0.0,130.8630,0.0,6,139.9600,1366.452
4,1084.6900,419.8800,16.7952,341.1525000,0.0,13.0863000000,0.0,0.0,0.0,0.0,...,34.9900000000,419.8800,0.0,34.9900000000,0.0,157.0356,0.0,11,349.9000,1366.452
5,1539.5600,209.9400,30.7912,524.8500000,0.0,13.0863000000,0.0,0.0,0.0,0.0,...,34.9900000000,209.9400,0.0,34.9900000000,0.0,78.5178,0.0,31,419.8800,1366.452
6,1189.6600,384.8900,86.7752,804.7700000,0.0,13.0863000000,0.0,0.0,0.0,0.0,...,34.9900000000,384.8900,0.0,34.9900000000,0.0,143.9493,0.0,44,209.9400,1366.452
7,1154.6700,1084.6900,123.1648,1049.7000000,0.0,13.0863000000,0.0,0.0,0.0,0.0,...,34.9900000000,1084.6900,0.0,34.9900000000,0.0,405.6753,0.0,34,384.8900,1366.452
8,1294.6300,1539.5600,95.1728,1242.1450000,0.0,13.0863000000,0.0,0.0,0.0,0.0,...,34.9900000000,1539.5600,0.0,34.9900000000,0.0,575.7972,0.0,33,1084.6900,1366.452
9,1014.7100,1189.6600,92.3736,1294.6300000,0.0,13.0863000000,0.0,0.0,0.0,0.0,...,34.9900000000,1189.6600,0.0,34.9900000000,0.0,444.9342,0.0,37,1539.5600,1366.452


#### Imputation

As a final step before modeling, all numeric columns should have missing values replaced by a number. This can be done by the `impute` transformation. If a linear or logistic regression, SVM or neural network algorithm was going to be applied, we may want to impute the mean or median. This could be done by passing 'mean' or 'median' in through the imputations dictionary.

As the modeling algoritm applied here is tree-based, we can simply impute and extreme value. All of the features created are non-negative or close to zero, so we will impute a very large negative number, *-999,999*.

In [67]:
imputation_dict = {'DISCOUNTAMOUNT_AVG': -999999,
                   'DISCOUNTAMOUNT_MAX': -999999,
                   'DISCOUNTAMOUNT_MIN': -999999,
                   'DISCOUNTAMOUNT_SUM': -999999,
                   'DISCOUNTPCT_AVG': -999999,
                   'DISCOUNTPCT_MAX': -999999,
                   'DISCOUNTPCT_MIN': -999999,
                   'DISCOUNTPCT_SUM': -999999,
                   'LAG_DISCOUNTAMOUNT_AVG_1': -999999,
                   'LAG_DISCOUNTAMOUNT_AVG_12': -999999,
                   'LAG_DISCOUNTAMOUNT_AVG_2': -999999,
                   'LAG_DISCOUNTAMOUNT_AVG_3': -999999,
                   'LAG_DISCOUNTPCT_AVG_1': -999999,
                   'LAG_DISCOUNTPCT_AVG_12': -999999,
                   'LAG_DISCOUNTPCT_AVG_2': -999999,
                   'LAG_DISCOUNTPCT_AVG_3': -999999,
                   'LAG_ORDERQUANTITY_SUM_1': -999999,
                   'LAG_ORDERQUANTITY_SUM_12': -999999,
                   'LAG_ORDERQUANTITY_SUM_2': -999999,
                   'LAG_ORDERQUANTITY_SUM_3': -999999,
                   'LAG_PRODUCTSTANDARDCOST_AVG_1': -999999,
                   'LAG_PRODUCTSTANDARDCOST_AVG_12': -999999,
                   'LAG_PRODUCTSTANDARDCOST_AVG_2': -999999,
                   'LAG_PRODUCTSTANDARDCOST_AVG_3': -999999,
                   'LAG_SALESAMOUNT_SUM_1': -999999,
                   'LAG_SALESAMOUNT_SUM_12': -999999,
                   'LAG_SALESAMOUNT_SUM_2': -999999,
                   'LAG_SALESAMOUNT_SUM_3': -999999,
                   'LAG_TAXAMT_SUM_1': -999999,
                   'LAG_TAXAMT_SUM_12': -999999,
                   'LAG_TAXAMT_SUM_2': -999999,
                   'LAG_TAXAMT_SUM_3': -999999,
                   'LAG_TOTALPRODUCTCOST_SUM_1': -999999,
                   'LAG_TOTALPRODUCTCOST_SUM_12': -999999,
                   'LAG_TOTALPRODUCTCOST_SUM_2': -999999,
                   'LAG_TOTALPRODUCTCOST_SUM_3': -999999,
                   'LAG_UNITPRICEDISCOUNTPCT_AVG_1': -999999,
                   'LAG_UNITPRICEDISCOUNTPCT_AVG_12': -999999,
                   'LAG_UNITPRICEDISCOUNTPCT_AVG_2': -999999,
                   'LAG_UNITPRICEDISCOUNTPCT_AVG_3': -999999,
                   'LAG_UNITPRICE_AVG_1': -999999,
                   'LAG_UNITPRICE_AVG_12': -999999,
                   'LAG_UNITPRICE_AVG_2': -999999,
                   'LAG_UNITPRICE_AVG_3': -999999,
                   'LAG_UNITPRICE_SUM_1': -999999,
                   'LAG_UNITPRICE_SUM_12': -999999,
                   'LAG_UNITPRICE_SUM_2': -999999,
                   'LAG_UNITPRICE_SUM_3': -999999,
                   'MEAN_ORDERQUANTITY_SUM_4': -999999,
                   'MEAN_SALESAMOUNT_SUM_4': -999999,
                   'ORDERQUANTITY_SUM': -999999,
                   'PRODUCTSTANDARDCOST_AVG': -999999,
                   'PRODUCTSTANDARDCOST_SUM': -999999,
                   'SALESAMOUNT_SUM': -999999,
                   'TAXAMT_SUM': -999999,
                   'TOTALPRODUCTCOST_AVG': -999999,
                   'TOTALPRODUCTCOST_SUM': -999999,
                   'UNITPRICEDISCOUNTPCT_AVG': -999999,
                   'UNITPRICEDISCOUNTPCT_MAX': -999999,
                   'UNITPRICEDISCOUNTPCT_MIN': -999999,
                   'UNITPRICEDISCOUNTPCT_SUM': -999999,
                   'UNITPRICE_AVG': -999999,
                   'UNITPRICE_SUM': -999999}

In [72]:
modelingds_imputed = modelingds_encoded.        (imputations=imputation_dict)

modelingds_imputed.order(order_by={'PRODUCTKEY':'ASC','ORDERWEEK':'ASC'}).preview()

,DISCOUNTAMOUNT_AVG,DISCOUNTAMOUNT_MIN,DISCOUNTPCT_AVG,LAG_ORDERQUANTITY_SUM_1,LAG_UNITPRICE_AVG_3,UNITPRICEDISCOUNTPCT_SUM,LAG_UNITPRICE_SUM_12,LAG_DISCOUNTAMOUNT_AVG_12,LAG_DISCOUNTPCT_AVG_12,LAG_PRODUCTSTANDARDCOST_AVG_1,...,UNITPRICEDISCOUNTPCT_MIN,DISCOUNTPCT_MAX,LAG_DISCOUNTAMOUNT_AVG_2,LAG_UNITPRICEDISCOUNTPCT_AVG_1,LAG_ORDERQUANTITY_SUM_2,LAG_TAXAMT_SUM_3,MEAN_ORDERQUANTITY_SUM_4,LAG_UNITPRICE_SUM_3,PRODUCTSTANDARDCOST_SUM,LAG_UNITPRICE_AVG_1
0,0.0,0.0,0.0,-999999,-999999.0000000000,0.0,-999999.0000,-999999.0,-999999.0,-999999.0000000000,...,0.0,0.0,-999999.0,-999999.0,-999999,-999999.0000,4.000,-999999.0000,52.3452,-999999.0000000000
1,0.0,0.0,0.0,4,-999999.0000000000,0.0,-999999.0000,-999999.0,-999999.0,13.0863000000,...,0.0,0.0,-999999.0,0.0,-999999,-999999.0000,7.000,-999999.0000,130.8630,34.9900000000
2,0.0,0.0,0.0,10,-999999.0000000000,0.0,-999999.0000,-999999.0,-999999.0,13.0863000000,...,0.0,0.0,0.0,0.0,4,-999999.0000,8.666,-999999.0000,157.0356,34.9900000000
3,0.0,0.0,0.0,12,34.9900000000,0.0,-999999.0000,-999999.0,-999999.0,13.0863000000,...,0.0,0.0,0.0,0.0,10,11.1968,8.000,139.9600,78.5178,34.9900000000
4,0.0,0.0,0.0,6,34.9900000000,0.0,-999999.0000,-999999.0,-999999.0,13.0863000000,...,0.0,0.0,0.0,0.0,12,27.9920,9.750,349.9000,143.9493,34.9900000000
5,0.0,0.0,0.0,11,34.9900000000,0.0,-999999.0000,-999999.0,-999999.0,13.0863000000,...,0.0,0.0,0.0,0.0,6,33.5904,15.000,419.8800,405.6753,34.9900000000
6,0.0,0.0,0.0,31,34.9900000000,0.0,-999999.0000,-999999.0,-999999.0,13.0863000000,...,0.0,0.0,0.0,0.0,11,16.7952,23.000,209.9400,575.7972,34.9900000000
7,0.0,0.0,0.0,44,34.9900000000,0.0,-999999.0000,-999999.0,-999999.0,13.0863000000,...,0.0,0.0,0.0,0.0,31,30.7912,30.000,384.8900,444.9342,34.9900000000
8,0.0,0.0,0.0,34,34.9900000000,0.0,-999999.0000,-999999.0,-999999.0,13.0863000000,...,0.0,0.0,0.0,0.0,44,86.7752,35.500,1084.6900,431.8479,34.9900000000
9,0.0,0.0,0.0,33,34.9900000000,0.0,-999999.0000,-999999.0,-999999.0,13.0863000000,...,0.0,0.0,0.0,0.0,34,123.1648,37.000,1539.5600,484.1931,34.9900000000


#### Train-test split

As this is a time-series problem, a random train-test split won't work as there will be leakage from observations near the end of the time frame in the training set to observations earlier than this in the test set. The way to avoid this problem is to perform the split based on the date. The transformation `train_test_split` can do this by passing the date columns through the parameter **order_by**.

In [73]:
modelingds_split = modelingds_imputed.train_test_split(order_by=['ORDERWEEK'], 
                                         train_percent=     )
    
modelingds_split.order(order_by={'PRODUCTKEY':'ASC', 'ORDERWEEK':'ASC'}).preview()

,DISCOUNTPCT_SUM,LAG_DISCOUNTAMOUNT_AVG_12,LAG_DISCOUNTPCT_AVG_12,UNITPRICE_AVG,LAG_TAXAMT_SUM_2,TOTALPRODUCTCOST_AVG,TOTALPRODUCTCOST_SUM,UNITPRICEDISCOUNTPCT_MIN,LAG_DISCOUNTAMOUNT_AVG_1,LAG_PRODUCTSTANDARDCOST_AVG_1,...,LAG_UNITPRICEDISCOUNTPCT_AVG_2,LAG_UNITPRICE_SUM_12,TAXAMT_SUM,DISCOUNTPCT_MIN,LAG_ORDERQUANTITY_SUM_2,LAG_TAXAMT_SUM_1,LAG_TOTALPRODUCTCOST_SUM_1,LAG_UNITPRICE_AVG_3,PRODUCTSTANDARDCOST_AVG,TT_SPLIT
0,0.0,-999999.0,-999999.0,34.9900000000,-999999.0000,13.0863000000,52.3452,0.0,-999999.0,-999999.0000000000,...,-999999.0,-999999.0000,11.1968,0.0,-999999,-999999.0000,-999999.0000,-999999.0000000000,13.0863000000,TRAIN
1,0.0,-999999.0,-999999.0,34.9900000000,-999999.0000,13.0863000000,130.8630,0.0,0.0,13.0863000000,...,-999999.0,-999999.0000,27.9920,0.0,-999999,11.1968,52.3452,-999999.0000000000,13.0863000000,TRAIN
2,0.0,-999999.0,-999999.0,34.9900000000,11.1968,13.0863000000,157.0356,0.0,0.0,13.0863000000,...,0.0,-999999.0000,33.5904,0.0,4,27.9920,130.8630,-999999.0000000000,13.0863000000,TRAIN
3,0.0,-999999.0,-999999.0,34.9900000000,27.9920,13.0863000000,78.5178,0.0,0.0,13.0863000000,...,0.0,-999999.0000,16.7952,0.0,10,33.5904,157.0356,34.9900000000,13.0863000000,TRAIN
4,0.0,-999999.0,-999999.0,34.9900000000,33.5904,13.0863000000,143.9493,0.0,0.0,13.0863000000,...,0.0,-999999.0000,30.7912,0.0,12,16.7952,78.5178,34.9900000000,13.0863000000,TRAIN
5,0.0,-999999.0,-999999.0,34.9900000000,16.7952,13.0863000000,405.6753,0.0,0.0,13.0863000000,...,0.0,-999999.0000,86.7752,0.0,6,30.7912,143.9493,34.9900000000,13.0863000000,TRAIN
6,0.0,-999999.0,-999999.0,34.9900000000,30.7912,13.0863000000,575.7972,0.0,0.0,13.0863000000,...,0.0,-999999.0000,123.1648,0.0,11,86.7752,405.6753,34.9900000000,13.0863000000,TRAIN
7,0.0,-999999.0,-999999.0,34.9900000000,86.7752,13.0863000000,444.9342,0.0,0.0,13.0863000000,...,0.0,-999999.0000,95.1728,0.0,31,123.1648,575.7972,34.9900000000,13.0863000000,TRAIN
8,0.0,-999999.0,-999999.0,34.9900000000,123.1648,13.0863000000,431.8479,0.0,0.0,13.0863000000,...,0.0,-999999.0000,92.3736,0.0,44,95.1728,444.9342,34.9900000000,13.0863000000,TRAIN
9,0.0,-999999.0,-999999.0,34.9900000000,95.1728,13.0863000000,484.1931,0.0,0.0,13.0863000000,...,0.0,-999999.0000,103.5704,0.0,34,92.3736,431.8479,34.9900000000,13.0863000000,TRAIN


#### Save Modeling Dataset

We can now save this modeling dataset so we can return to it in the future.

In [74]:
modeling = rasgo.        .dataset(dataset=modelingds_split,
                                 name="WKSP: AdventureWorks: Sales Forecast Modeling",
                                 description="Modeling dataset for Internet Sales Forecasting")
modeling

Dataset(id=2395, name=WKSP: AdventureWorks: Sales Forecast Modeling, status=published, description=Modeling dataset for Internet Sales Forecasting)

We can examine this dataset on Rasgo by clicking the link below

In [76]:
print(f"https://app.rasgoml.com/datasets/{modeling.id}")

https://app.rasgoml.com/datasets/2395


Capture this dataset ID for use in prediction.

In [87]:
ds_id =          .id

2395

### Modeling

We are now ready to build the model. First, get the modeling data from Rasgo using `to_df`.

In [77]:
df = modeling.    ().reset_index(drop=True)
df = df[~df.ORDERWEEK.isna()]

Check for numeric datatypes and convert the numeric ones to floats.

In [78]:
for c in df.select_dtypes(exclude=[np.number]).columns:
    if c not in [           , 'TT_SPLIT']:
        df[c] = pd.to_numeric(df[c])

Eliminate the last week of data as there is no target.

In [79]:
df = df[~df.           .isna()]

#### Train the model

First, split the data using the TT_SPLIT column.

In [80]:
df_train = df[df['TT_SPLIT'] == 'TRAIN'].drop(columns=['TT_SPLIT', 'ORDERWEEK'])
df_test = df[df['TT_SPLIT'] ==        ].drop(columns=['TT_SPLIT', 'ORDERWEEK'])

In [81]:
y_train = df_train['TARGET_SALESAMOUNT']
X_train = df_train.drop(columns=['TARGET_SALESAMOUNT'])
y_test = df_test['TARGET_SALESAMOUNT']
X_test = df_test.drop(columns=['TARGET_SALESAMOUNT'])

#### Fit the model

For illustration purposes, we are just fitting the model with a single set of parameters. In general, you should optimize the hyperparameters before building the final model. That process is beyond the scope of this document.

In [82]:
model = xgb.XGBRegressor(n_estimators=    ,
                         max_depth=5,
                         eta=0.01,
                         random_state=1066,
                         subsample=0.7,
                         colsample_bytree=0.7)

model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, enable_categorical=False,
             eta=0.01, gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.00999999978,
             max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=1066,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.7,
             tree_method='exact', validate_parameters=1, verbosity=None)

#### Check the performance

In [ ]:
model.predict(X_test)

In [83]:
rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
rmse

4836.937497073011

### Predict on new data

Since our feature engineering was saved in Rasgo, as new data enters the system, it will automatically be prepared for modeling. We can just pull the data in question and make a prediction on it.

In this case, if we are making these predictions each week, we can just pull the most recent week. In this particular data, that is '*2014-01-19*'.

#### Pull the data

Use to_df to grab the data from this date. We have several columns not needed in the model, so we will drop those as well.

In [92]:
predictdf = rasgo.get.dataset(ds_id).to_df(filters=["ORDERWEEK = '2014-01-19'"])
for c in predictdf.select_dtypes(exclude=[np.number]).columns:
    if c not in [          , 'TT_SPLIT']:
        predictdf[c] = pd.to_numeric(predictdf[c])
predictdf.head()

,DISCOUNTAMOUNT_MIN,DISCOUNTPCT_MAX,LAG_TOTALPRODUCTCOST_SUM_1,LAG_ORDERQUANTITY_SUM_12,LAG_TAXAMT_SUM_3,ORDERWEEK,LAG_UNITPRICE_AVG_3,DISCOUNTAMOUNT_SUM,LAG_DISCOUNTAMOUNT_AVG_1,LAG_DISCOUNTPCT_AVG_12,...,DISCOUNTAMOUNT_AVG,LAG_DISCOUNTPCT_AVG_2,LAG_TAXAMT_SUM_1,LAG_TOTALPRODUCTCOST_SUM_2,MEAN_SALESAMOUNT_SUM_4,LAG_DISCOUNTPCT_AVG_1,LAG_UNITPRICE_AVG_2,LAG_TOTALPRODUCTCOST_SUM_3,SALESAMOUNT_SUM,TT_SPLIT
0,0.0,0.0,157.0578,9,39.1944,2014-01-19,69.99,0.0,0.0,0.0,...,0.0,0.0,33.5952,52.3526,332.4525,0.0,69.99,183.2341,279.96,TEST
1,0.0,0.0,41.5723,4,12.9576,2014-01-19,53.99,0.0,0.0,0.0,...,0.0,0.0,4.3192,83.1446,94.4825,0.0,53.99,124.7169,53.99,TEST
2,0.0,0.0,179.5200,16,9.6000,2014-01-19,120.00,0.0,0.0,0.0,...,0.0,0.0,38.4000,314.1600,390.0000,0.0,120.00,44.8800,120.00,TEST
3,0.0,0.0,261.7260,56,53.1848,2014-01-19,34.99,0.0,0.0,0.0,...,0.0,0.0,55.9840,379.5027,769.7800,0.0,34.99,248.6397,699.80,TEST
4,0.0,0.0,6.7246,7,5.7536,2014-01-19,8.99,0.0,0.0,0.0,...,0.0,0.0,1.4384,13.4492,33.7125,0.0,8.99,26.8984,8.99,TEST


Now use the model to get the sales forecast. We will create a dataframe to hold the predictions then drop the columns not needed by the model before making the prediction.

In [93]:
salesforecastdf = predictdf[['PRODUCTKEY', 'ORDERWEEK']].copy()
salesforecastdf[            ] = model.predict(predictdf.drop(columns=['TT_SPLIT', 'ORDERWEEK', 'TARGET_SALESAMOUNT']))
salesforecastdf

,PRODUCTKEY,ORDERWEEK,forecast
0,475,2014-01-19,250.618988
1,490,2014-01-19,135.491135
2,483,2014-01-19,331.826996
3,222,2014-01-19,707.059570
4,482,2014-01-19,113.257957
5,228,2014-01-19,239.069534
6,479,2014-01-19,116.006882
7,538,2014-01-19,207.328751
8,480,2014-01-19,113.257957
9,463,2014-01-19,116.421211
